<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR02/ASR_speechgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@markdown Setup

!pip install gtts
import gtts
from gtts import gTTS
import pandas as pd
from IPython.display import Audio

In [ ]:
#@markdown Define etts(), 'sample.mp3'
def etts(mytext):
    tts = gTTS(mytext)
    tts.save('sample.wav')
    return Audio('sample.wav', autoplay=True)
def ktts(mytext):
    tts = gTTS(mytext, lang = "ko")
    tts.save('sample.wav')
    return Audio('sample.wav', autoplay=True)

In [ ]:
txt = "The programs about detectives and hospitals are my favorites."
ktts(txt)
Audio('sample.wav')

In [ ]:
etts(txt)
Audio('sample.wav')

In [ ]:
#@markdown 🎯Install SR tool
%%capture
!pip install git+https://github.com/openai/whisper.git 

import whisper
model = whisper.load_model('base.en') 
model2 = whisper.load_model('medium.en')

Speech file upload

+ Ksent.wav  
+ Esent.wav  

In [ ]:
#@markdown 🎯 Create a file list as a dataframe (df1)
import os
import pandas as pd

filepath = input("Type the full file path to locate audio files. (e.g., /content/HE) ")
dir_path = str(filepath)
dir_files = os.listdir(dir_path)
str1 = 'wav'
flist = []

for i in range(0, len(dir_files)):
  str2 = dir_files[i]
  if str1 in str2:
    flist.append(str2)

flist = sorted(flist)

df1 = pd.DataFrame()
n = len(flist)
nt = n + 1
fn = range(1, nt)
df1['ID'] = fn
df1['Filename'] = flist

# print(df.to_string(index=False))
df1.head()

In [ ]:
#@markdown 🎯Install {autotime} to measure runtime (From here, runtime appears automatically.)
%%capture
!pip install ipython-autotime
%load_ext autotime

In [ ]:
df1.head()

In [ ]:
#@markdown 🎯 Testing ASR (single file): Type a number between 1~19
rname = input("Type ID: ")
ind = int(rname) - 1
myf = df1['Filename'][ind]
result = model.transcribe(myf, language="en",fp16=False)
print('Filename: %s'%myf)
print('='*30)

print("Speech-to-text (recognized): %s"%result["text"])  

In [ ]:
#@markdown =======================================================================
#@markdown ## 🎯Processing multiple files (19) and saving it as **df2**

#@markdown ## **Note:** Current directory should be the folder having the audio files (e.g., spk01)
#@markdown =======================================================================

import os
a = !pwd
print(a)

checkdir = input("Need to change current directory? (y for 'yes', n for 'no'")

if checkdir == 'y':

  filepath = input("Change current directory to... /content/destination/")
  os.chdir(filepath)
  b = !pwd
  print("Current directory changed to: %s"%b)
else:
  print('Okay, proceed ASR.')

import time
import pandas as pd

fname = []
rt = []
rectext = []
df2 = pd.DataFrame()

def measure_time(function):
  start = time.time()
  function()
  end = time.time()
  return end - start

nfiles = len(df1['Filename']) #19

for i in range(0, nfiles):
  rname = df1['ID'][i]
  ind = int(rname)
  myf = df1['Filename'][i]
  fname.append(myf)

  def code_to_measure():
# your code here  
    result = model.transcribe(myf, language="en",fp16=False)
    print('='*30)
    print("Speech-to-text (recognized): %s"%result['text']) 
    recresult = result['text']
    rectext.append(recresult)

  runtime = round(measure_time(code_to_measure),3)
  rt.append(str(runtime))

  print(f"Runtime: {runtime} seconds")
  print('Filename: %s'%myf)

df2['Filename'] = fname
df2['Runtime'] = rt
df2['Recognized'] = rectext

df2.head()

In [ ]:
import pandas as pd

sent = ["The programs about detectives and hospitals are my favorites","The programs about detectives and hospitals are my favorites","The programs about detectives and hospitals are my favorites"]
sn = ["SK","SE","HS"]

df = pd.DataFrame()
df['SN'] = sn
df['Sentence'] = sent
df

In [ ]:
# Add original text to the result file

df2['Sentence'] = df['Sentence']
df2['SN'] = df['SN']

df2 = df2[['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']]
cols = list(df2.columns.values)

df2.head()

In [ ]:
#@markdown + Install and import libraries
%%capture
!pip install Levenshtein
import Levenshtein as lev
import numpy as np
import pandas as pd

In [ ]:
#@markdown + Calculate WER and WER_lev and add columns to df
# import numpy as np
# import pandas as pd
# import Levenshtein as lev
#@markdown + Column names 'Original' and 'Recognized' for comparison
def calculate_wer_word_level(s1, s2):
    """
    Calculation of WER with Dynamic Time Warping algorithm at word level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # preparing the matrix for the Dynamic Time Warping algorithm
    dtw_matrix = np.zeros((len(s1_words)+1, len(s2_words)+1))
    for i in range(len(s1_words)+1):
        for j in range(len(s2_words)+1):
            if i == 0:
                dtw_matrix[0][j] = j
            elif j == 0:
                dtw_matrix[i][0] = i

    # Dynamic Time Warping algorithm
    for i in range(1, len(s1_words)+1):
        for j in range(1, len(s2_words)+1):
            cost = 0 if s1_words[i-1] == s2_words[j-1] else 1
            dtw_matrix[i][j] = cost + min([dtw_matrix[i-1][j], dtw_matrix[i][j-1], dtw_matrix[i-1][j-1]])

    wer = dtw_matrix[len(s1_words)][len(s2_words)] / len(s1_words)

    return wer

def calculate_wer_lev(s1, s2):
    """
    Calculation of WER with Levenshtein distance at character level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # calculating Levenshtein distance
    edit_distance = lev.distance(' '.join(s1_words), ' '.join(s2_words))

    # normalizing by the length of the original sentence (s1)
    wer = edit_distance / len(s1_words)

    return wer

# Assuming df is your DataFrame and it has columns 'Sentence' and 'Recognized'
wer_values_word_level = df2.apply(lambda row: calculate_wer_word_level(row['Sentence'], row['Recognized']), axis=1)
wer_values_lev = df2.apply(lambda row: calculate_wer_lev(row['Sentence'], row['Recognized']), axis=1)

# Adding WER values as new columns to the DataFrame
df2['WER'] = wer_values_word_level
df2['WER_lev'] = wer_values_lev

df2


---

In [ ]:
#@markdown   [1] Make a new folder: type a new folder name (e.g., myaudio)
import os

folder_name = input("Type a name for the new folder.")

if not os.path.exists(folder_name):
  os.makedirs(folder_name)
  print(f"A new folder name '{folder_name}' has been created.")
else:
  print(f"{folder_name} already exists.")

In [ ]:
#@markdown 🎯Mount Google drive and list files (e.g., "asrdata" folder in my case)
from google.colab import drive 
import os

drive.mount('/content/drive')

mydir = input("Type the file directory in your google drive: e.g., asrdata  ")
!ls "/content/drive/MyDrive/{mydir}"
!pwd

In [ ]:
#@markdown 🎯 Unmount Google drive (clearing): optional

from google.colab import drive
drive.flush_and_unmount()